<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/managed/vectaraDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Managed Index with Zilliz Cloud Pipelines

[Zilliz Cloud Pipelines](https://docs.zilliz.com/docs/pipelines) is a scalable API service for retrieval. You can use Zilliz Cloud Pipelines as managed index in `llama-index`. This service can transform documents into vector embeddings and store them in Zilliz Cloud for effective semantic search.

## Setup

1. Install llama-index dependencies

In [ ]:
%pip install llama-index-indices-managed-zilliz

In [ ]:
%pip install llama-index

2. Configure credentials of your [Zilliz Cloud](https://cloud.zilliz.com/signup?utm_source=twitter&utm_medium=social%20&utm_campaign=2023-12-22_social_pipeline-llamaindex_twitter) accounts.

In [ ]:
from getpass import getpass

ZILLIZ_PROJECT_ID = getpass("Enter your Zilliz Project ID:")
ZILLIZ_CLUSTER_ID = getpass("Enter your Zilliz Cluster ID:")
ZILLIZ_TOKEN = getpass("Enter your Zilliz API Key:")

> [Find your OpenAI API key](https://beta.openai.com/account/api-keys)
>
> [Find your Zilliz Cloud credentials](https://docs.zilliz.com/docs/on-zilliz-cloud-console)

## Indexing documents

> It is optional to add metadata for each document. The metadata can be used to filter doc data during retrieval.

### From Signed URL

Zilliz Cloud Pipelines accepts files from AWS S3 and Google Cloud Storage. You can generate a presigned url from the Object Storage and use `from_document_url()` to ingest the file. It can automatically index the document and store the doc chunks as vectors on Zilliz Cloud.

In [ ]:
from llama_index.indices.managed.zilliz import ZillizCloudPipelineIndex

# Create pipelines: skip this step if you have prepared valid pipelines
pipeline_ids = ZillizCloudPipelineIndex.create_pipelines(
    project_id=ZILLIZ_PROJECT_ID,
    cluster_id=ZILLIZ_CLUSTER_ID,
    api_key=ZILLIZ_TOKEN,
    data_type="doc",
    collection_name="zcp_llamalection_doc",  # change this value will customize collection name
    metadata_schema={"user_id": "VarChar"},
)
print(pipeline_ids)

{'INGESTION': 'pipe-d639f220f27320e2e381de', 'SEARCH': 'pipe-47bd43fe8fd54502874a08', 'DELETION': 'pipe-bd434c99e064282f1a28e8'}


In [ ]:
zcp_doc_index = ZillizCloudPipelineIndex.from_document_url(
    # a public or pre-signed url of a file stored on AWS S3 or Google Cloud Storage
    url="https://publicdataset.zillizcloud.com/milvus_doc.md",
    pipeline_ids=pipeline_ids,
    api_key=ZILLIZ_TOKEN,
    metadata={
        "user_id": "user-001"
    },  # optional, which can be used for filtering
)

# # Delete docs by doc name
# zcp_doc_index.delete_by_expression(expression="doc_name == 'milvus_doc_22.md'")

### From Document Nodes

Zilliz Cloud Pipelines support text as data input as well. The following example prepares data with a sample document node.

In [ ]:
from llama_index.core import Document
from llama_index.indices.managed.zilliz import ZillizCloudPipelineIndex

# prepare documents
documents = [Document(text="The number that is being searched for is ten.")]

# create pipelines: skip this step if you have prepared valid pipelines
pipeline_ids = ZillizCloudPipelineIndex.create_pipelines(
    project_id=ZILLIZ_PROJECT_ID,
    cluster_id=ZILLIZ_CLUSTER_ID,
    api_key=ZILLIZ_TOKEN,
    data_type="text",
    collection_name="zcp_llamalection_text",  # change this value will customize collection name
)
print(pipeline_ids)

{'INGESTION': 'pipe-2bbab10f273a57eb987024', 'SEARCH': 'pipe-e1914a072ec5e6f83e446a', 'DELETION': 'pipe-72bbabf273a51af0b0c447'}


In [ ]:
zcp_text_index = ZillizCloudPipelineIndex.from_documents(
    # a public or pre-signed url of a file stored on AWS S3 or Google Cloud Storage
    documents=documents,
    pipeline_ids=pipeline_ids,
    api_key=ZILLIZ_TOKEN,
)

## Working as Query Engine

To conduct semantic search with `ZillizCloudPipelineIndex`, you can use it `as_query_engine()` by specifying a few parameters:
- **search_top_k**: How many text nodes/chunks to retrieve. Optional, defaults to `DEFAULT_SIMILARITY_TOP_K` (2).
- **filters**: Metadata filters. Optional, defaults to None.
- **output_metadata**: What metadata fields to return with the retrieved text node. Optional, defaults to [].

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key:")

In [ ]:
query_engine = zcp_doc_index.as_query_engine(search_top_k=3)

Then the query engine is ready for Semantic Search or Retrieval Augmented Generation with Milvus 2.3 documents:

- **Retrieve** (Semantic search powered by Zilliz Cloud Pipelines):

In [ ]:
question = "Can users delete entities by filtering non-primary fields?"
retrieved_nodes = query_engine.retrieve(question)
print(retrieved_nodes)

[NodeWithScore(node=TextNode(id_='449755997496672548', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Delete Entities\nThis topic describes how to delete entities in Milvus.  \nMilvus supports deleting entities by primary key or complex boolean expressions. Deleting entities by primary key is much faster and lighter than deleting them by complex boolean expressions. This is because Milvus executes queries first when deleting data by complex boolean expressions.  \nDeleted entities can still be retrieved immediately after the deletion if the consistency level is set lower than Strong.\nEntities deleted beyond the pre-specified span of time for Time Travel cannot be retrieved again.\nFrequent deletion operations will impact the system performance.  \nBefore deleting entities by comlpex boolean expressions, make sure the collection has been loaded.\nDeleting entities by complex boolean expressions is not an atomic ope

- **Query** (RAG powered by Zilliz Cloud Pipelines as retriever and OpenAI's LLM):

In [ ]:
response = query_engine.query(question)
print(response.response)

Users can delete entities by filtering non-primary fields using complex boolean expressions in Milvus.


## Multi-Tenancy

With the tenant-specific value (eg. user id) as metadata, the managed index is able to achieve multi-tenancy by applying metadata filters.

By specifying metadata value, each document is tagged with the tenant-specific field at ingestion.

In [ ]:
zcp_doc_index._insert_doc_url(
    url="https://publicdataset.zillizcloud.com/milvus_doc_22.md",
    metadata={"user_id": "user_002"},
)

{'token_usage': 984, 'doc_name': 'milvus_doc_22.md', 'num_chunks': 3}

Then the managed index is able to build a query engine for each tenant by filtering the tenant-specific field.

In [ ]:
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters

query_engine_for_user_002 = zcp_doc_index.as_query_engine(
    search_top_k=3,
    filters=MetadataFilters(
        filters=[ExactMatchFilter(key="user_id", value="user_002")]
    ),
    output_metadata=["user_id"],  # optional, display user_id in outputs
)

> Change `filters` to build query engines with different conditions.

In [ ]:
question = "Can I delete entities by filtering non-primary fields?"

# search_results = query_engine_for_user_002.retrieve(question)
response = query_engine_for_user_002.query(question)
print(response.response)

Milvus only supports deleting entities by primary key filtered with boolean expressions. Other operators can be used only in query or scalar filtering in vector search.
